In [1]:
import ctypes

class DynamicArray:
    def __init__(self):
        self._n = 0  # 当前元素数量
        self._capacity = 1  # 当前容量
        self._A = self._make_array(self._capacity)

    def __len__(self):
        return self._n

    def __getitem__(self, k):
        if 0 <= k < self._n:
            return self._A[k]
        raise IndexError('Index out of bounds')

    def append(self, obj):
        if self._n == self._capacity:
            self._resize(2 * self._capacity)  # 容量不够，扩容
        self._A[self._n] = obj
        self._n += 1

    def _resize(self, new_cap):
        B = self._make_array(new_cap)
        for k in range(self._n):
            B[k] = self._A[k]
        self._A = B
        self._capacity = new_cap

    def _make_array(self, c):
        return (c * ctypes.c_int)()

    def __str__(self):
        return '[' + ', '.join(str(self._A[i]) for i in range(self._n)) + ']'

In [2]:
arr = DynamicArray()
for i in range(10):
    arr.append(i)
print(arr)


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


## Problem

> Given a dynamic table (see Section 17.4) that doubles in size when it needs more space, find the **amortized runtime** for inserting `n` elements:
> - a) Use the **Aggregate Method**
> - b) Use the **Accounting Method**

---

## Solution

### a) Aggregate Method

**Idea:**  
We analyze the total cost of inserting `n` elements into a dynamic array that doubles in size when full.

- Regular insert: cost = 1
- When resizing: we also copy all existing elements

#### Capacity doubling:
The capacity grows like: 1 → 2 → 4 → 8 → ...  
Each time the array is full, it resizes to double the size and copies all existing elements.

**Example for `n = 10`:**

| Insertion | Resizing? | Copy Cost |
|-----------|-----------|-----------|
| 1         | ✅         | 0         |
| 2         | ✅         | 1         |
| 3         |           | 0         |
| 4         | ✅         | 2         |
| 5–7       |           | 0         |
| 8         | ✅         | 4         |
| 9–10      |           | 0         |

Copy cost is: 1 + 2 + 4 + 8 + ... ≤ 2n - 1  
Insert cost is: `n`  
Total cost ≤ `n + 2n` = **3n**

#### Amortized Time:
$$
\frac{T(n)}{n} \leq \frac{3n}{n} = \boxed{O(1)}
$$

---

### b) Accounting Method

**Idea:**  
We assign an artificial (amortized) cost to each operation to "prepay" for future expensive operations.

#### Strategy:
- Charge each insertion **3 units**
- Spend:
  - 1 unit to insert the element
  - 2 units saved ("banked") to pay for future copy operations

#### Why it works:
- When we double the size, we need to copy all existing elements
- Each of those elements has saved 2 units, which is enough to pay for being copied

#### Amortized Time:
Every insert is charged a constant cost →  
$$
\boxed{O(1)} \text{ amortized time per insert}
$$

---

## Conclusion

Using both the **Aggregate** and **Accounting** methods, the amortized cost per insertion into a dynamic array that doubles its size is:

$$
\boxed{O(1)}
$$

This shows that dynamic arrays are efficient for append operations, even with occasional resizing.